# Task 1 – Stationen, Stops (planned) und Stops (changed)

Dieser Notebook fasst die vorherigen drei Task1-Notebooks zusammen.

In [1]:
# Notebook setup: allow imports from ../src
import sys
from pathlib import Path

repo_root = Path("..").resolve()
src_path = (repo_root / "src").as_posix()
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print("Using repo root:", repo_root)


Using repo root: C:\CodingProjects\DIA\DBahn-berlin\repo


In [ ]:
from src.timetable_etl.config import Settings
from src.timetable_etl.db import connect

settings = Settings.from_env()
print(settings)
conn = connect(settings)

Settings(db_host='localhost', db_name='postgres', db_user='postgres', db_password='1234', station_json_path=WindowsPath('C:/CodingProjects/DIA/DBahn-berlin/repo/station_data.json'), planned_archives_path=WindowsPath('C:/CodingProjects/DIA/DBahn-berlin/repo/timetables'), changes_archives_path=WindowsPath('C:/CodingProjects/DIA/DBahn-berlin/repo/timetable_changes'), archive_pattern='*.tar.gz', timezone='Europe/Berlin', match_threshold=0.85, ambiguity_delta=0.02, planned_batch_size=5000, change_batch_size=50000)


## 1) Stationen importieren (station_data.json → table `stationen`)

In [ ]:
from src.timetable_etl.stations import import_stationen

n = import_stationen(conn, settings.station_json_path)
print("Upserted station rows:", n)

Upserted station rows: 133


## 2) Stops planned importieren (timetables/*.tar.gz → table `stops`)

In [ ]:
from src.timetable_etl.stops_planned import import_stops_from_archives

planned_res = import_stops_from_archives(
    conn,
    settings.planned_archives_path,
    pattern=settings.archive_pattern,
    timezone=settings.timezone,
    match_threshold=settings.match_threshold,
    ambiguity_delta=settings.ambiguity_delta,
    batch_size=settings.planned_batch_size,
)
planned_res

{'total_seen_stops': 2104080,
 'total_upserted': 2094318,
 'unmatched_station_names': set(),
 'ambiguous_station_names': set(),
 'match_cache_size': 131}

## 3) Stops changed anwenden (timetable_changes/*.tar.gz → Update auf `stops`)

In [ ]:
from src.timetable_etl.stops_changed import process_change_archives

changed_res = process_change_archives(
    conn,
    settings.changes_archives_path,
    pattern=settings.archive_pattern,
    batch_size=settings.change_batch_size,
)
changed_res


{'change_files_processed': 540668,
 'stop_updates_staged': 1346664,
 'rows_updated_sql': 953744}

In [6]:
conn.close()
print("Done.")


Done.
